In [2]:
import pandas as pd
pd.set_option('display.max_rows', 10)

description_col = 'Security Description'
security_col = 'Security No.'
symbol_col = 'Symbol'
exchange_col = 'Exchange'
currency_col = 'Currency'
holding_col = 'Holding Jun24'
value_col = 'Market Value'
percent_col = '% of Fund'
jpm_sp_col = 'SP JPM GBP'
jun24_sp_col = 'SP Jun24 lc'
today_sp_col = 'SP Today lc'

In [3]:
jema_symbols = pd.read_csv('./jema_may_jun24_symbols.csv').dropna(subset=[symbol_col])
# jema_symbols[holding_col] = jema_symbols[holding_col].str.replace(',', '').astype(float)
jema_symbols.reset_index(inplace=True, drop=True)
jema_symbols

,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24
0,THE SAUDI NATIONAL BANK,BSHYYN1,1180,TADAWUL,SAR,1,81271.00
1,AL RAJHI BANK COMMON STOCK SAR 10,B12LZH9,1120,TADAWUL,SAR,1,32585.00
2,FIRSTRAND LTD COMMON STOCK ZAR 1,6606996,FSR,JSE,ZAR,1,164763.00
3,JPM GBP LIQUIDITY LVNAV X (DIST.),5819115,GBPGBP,FX_IDC,GBP,1,514330.73
4,STANDARD BANK GROUP LTD COMMON STOCK ZAR 10,B030GJ7,SBK,JSE,ZAR,1,47354.00
...,...,...,...,...,...,...,...
97,TATNEFT PJSC TOTAL,NaN,TATN,ALOR,RUB,1,1301425.00
98,TCS GROUP HOLDING PLC GDR,0H6380S,TCSG,ALOR,RUB,1,75226.00
99,VTB BANK PJSC COMMON STOCK RUB 0.01,0H5837S,VTBR,ALOR,RUB,1,4246655.00
100,X 5 RETAIL GROUP NV-REGS GDR,0H6365S,FIVE,ALOR,RUB,1,275182.00


In [4]:
jema_data = pd.read_excel('./jpm-emerging-europe-middle-east-afria-disclosure.xlsx', sheet_name='June 2024', skiprows=9, usecols=range(5))
jema_data = jema_data.dropna()
jema_data = jema_data.drop(columns=[description_col, 'Holding', percent_col])
jema_data[value_col] = jema_data[value_col].astype(float)
jema_data

,Market Value,Security No.
1,628932.02,BSHYYN1
2,559299.16,B12LZH9
3,548770.63,6606996
4,514330.73,5819115
5,432459.13,B030GJ7
...,...,...
110,14658.68,0H5808S
111,4294.55,0H6372S
112,1900.14,0H6380S
113,1156.81,0H6365S


In [5]:
symbols_with_jun_vals = pd.merge(jema_symbols, jema_data, on=security_col, how='left')

holdings_to_sum_vals = {
    "LKOH": ("2H6442S", "0H5797S"),
    "GAZP": ("0H5796S", "0H6364S"),
    "SBER": ("0H5856S", "0H5855S"),
    "NVTK": ("2H6464S", "0H5828S"),
    "ROSN": ("0H5827S", "2H7674S"),
    "TATN": ("0H5822S", "0H5816S"),
}

for symbol, (sec_no_1, sec_no_2) in holdings_to_sum_vals.items():
    val_sum = jema_data.loc[jema_data[security_col] == sec_no_1][value_col].item() + jema_data.loc[jema_data[security_col] == sec_no_2][value_col].item()
    symbols_with_jun_vals.loc[symbols_with_jun_vals[symbol_col] == symbol, value_col] = val_sum
    display(symbols_with_jun_vals.loc[symbols_with_jun_vals[symbol_col] == symbol])


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24,Market Value
63,LUKOIL PJSC TOTAL,NaN,LKOH,ALOR,RUB,1,624696.0,300684.88


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24,Market Value
36,GAZPROM PJSC TOTAL,NaN,GAZP,ALOR,RUB,1,19591077.0,226545.49


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24,Market Value
94,SBERBANK OF RUSSIA PJSC TOTAL,NaN,SBER,ALOR,RUB,1,16810961.0,211058.6


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24,Market Value
89,NOVATEK PJSC TOTAL,NaN,NVTK,ALOR,RUB,1,1372062.0,160222.55


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24,Market Value
92,ROSNEFT OIL CO PJSC TOTAL,NaN,ROSN,ALOR,RUB,1,3719692.0,107524.32


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24,Market Value
97,TATNEFT PJSC TOTAL,NaN,TATN,ALOR,RUB,1,1301425.0,44357.4


In [6]:
symbols_with_jun_vals[jpm_sp_col] = symbols_with_jun_vals[value_col] / symbols_with_jun_vals[holding_col]
symbols_with_jun_vals

,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24,Market Value,SP JPM
0,THE SAUDI NATIONAL BANK,BSHYYN1,1180,TADAWUL,SAR,1,81271.00,628932.02,7.738702
1,AL RAJHI BANK COMMON STOCK SAR 10,B12LZH9,1120,TADAWUL,SAR,1,32585.00,559299.16,17.164314
2,FIRSTRAND LTD COMMON STOCK ZAR 1,6606996,FSR,JSE,ZAR,1,164763.00,548770.63,3.330667
3,JPM GBP LIQUIDITY LVNAV X (DIST.),5819115,GBPGBP,FX_IDC,GBP,1,514330.73,514330.73,1.000000
4,STANDARD BANK GROUP LTD COMMON STOCK ZAR 10,B030GJ7,SBK,JSE,ZAR,1,47354.00,432459.13,9.132473
...,...,...,...,...,...,...,...,...,...
97,TATNEFT PJSC TOTAL,NaN,TATN,ALOR,RUB,1,1301425.00,44357.40,0.034084
98,TCS GROUP HOLDING PLC GDR,0H6380S,TCSG,ALOR,RUB,1,75226.00,1900.14,0.025259
99,VTB BANK PJSC COMMON STOCK RUB 0.01,0H5837S,VTBR,ALOR,RUB,1,4246655.00,39977.21,0.009414
100,X 5 RETAIL GROUP NV-REGS GDR,0H6365S,FIVE,ALOR,RUB,1,275182.00,1156.81,0.004204


In [7]:
symbols_with_jun_vals.to_csv(path_or_buf="../data/jema_symbols_with_vals_jun24.csv")

In [8]:
import time

from tvDatafeed import TvDatafeed, Interval

tv = TvDatafeed()

for index, row in symbols_with_jun_vals.iterrows():

    # time.sleep(10)

    symbol = row.Symbol
    exchange = row.Exchange

    hist = tv.get_hist(
        symbol=symbol,
        exchange=exchange,
        interval=Interval.in_daily,
        n_bars=100
    )

    try:
        jun3124_sp = hist.resample('ME').last().loc['2024-06'].close.item()
        sp_now = hist.iloc[-1].close.item()
        
        symbols_with_jun_vals.loc[index, jun24_sp_col] = jun3124_sp
        symbols_with_jun_vals.loc[index, today_sp_col] = sp_now
    except:
        print(f"Failed: {symbol} - {exchange}")

symbols_with_jun_vals

you are using nologin method, data you access may be limited


Failed: FIVE - ALOR


,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24,Market Value,SP JPM,SP Jun24,SP Today
0,THE SAUDI NATIONAL BANK,BSHYYN1,1180,TADAWUL,SAR,1,81271.00,628932.02,7.738702,36.25,36.849998
1,AL RAJHI BANK COMMON STOCK SAR 10,B12LZH9,1120,TADAWUL,SAR,1,32585.00,559299.16,17.164314,80.00,89.099998
2,FIRSTRAND LTD COMMON STOCK ZAR 1,6606996,FSR,JSE,ZAR,1,164763.00,548770.63,3.330667,7693.00,8446.000000
3,JPM GBP LIQUIDITY LVNAV X (DIST.),5819115,GBPGBP,FX_IDC,GBP,1,514330.73,514330.73,1.000000,1.00,1.000000
4,STANDARD BANK GROUP LTD COMMON STOCK ZAR 10,B030GJ7,SBK,JSE,ZAR,1,47354.00,432459.13,9.132473,21083.00,23642.000000
...,...,...,...,...,...,...,...,...,...,...,...
97,TATNEFT PJSC TOTAL,NaN,TATN,ALOR,RUB,1,1301425.00,44357.40,0.034084,699.90,603.100000
98,TCS GROUP HOLDING PLC GDR,0H6380S,TCSG,ALOR,RUB,1,75226.00,1900.14,0.025259,2968.50,2628.000000
99,VTB BANK PJSC COMMON STOCK RUB 0.01,0H5837S,VTBR,ALOR,RUB,1,4246655.00,39977.21,0.009414,105.70,100.180000
100,X 5 RETAIL GROUP NV-REGS GDR,0H6365S,FIVE,ALOR,RUB,1,275182.00,1156.81,0.004204,NaN,NaN


In [128]:
symbol= 'KOMB'
exchange = 'PSECZ'

hist = tv.get_hist(
    symbol=symbol,
    exchange=exchange,
    interval=Interval.in_daily,
    n_bars=100
)

display(hist)

jun3124_sp = hist.resample('ME').last().loc['2024-06'].close.item()
sp_now = hist.iloc[-1].close.item()

print(jun3124_sp)
print(sp_now)


,symbol,open,high,low,close,volume
datetime,,,,,,
2024-03-27 08:00:00,PSECZ:KOMB,827.0,833.5,823.5,832.5,143503.0
2024-03-28 08:00:00,PSECZ:KOMB,835.0,844.0,833.5,840.0,182780.0
2024-04-02 08:00:00,PSECZ:KOMB,839.5,851.0,838.0,851.0,250944.0
2024-04-03 08:00:00,PSECZ:KOMB,849.5,850.0,838.5,849.5,188025.0
2024-04-04 08:00:00,PSECZ:KOMB,850.0,855.0,848.5,855.0,119209.0
...,...,...,...,...,...,...
2024-08-14 08:00:00,PSECZ:KOMB,758.0,766.0,755.0,760.5,55182.0
2024-08-15 08:00:00,PSECZ:KOMB,762.0,767.5,758.0,762.0,109670.0
2024-08-16 08:00:00,PSECZ:KOMB,769.0,769.0,763.0,764.5,61615.0


782.5
766.0


In [9]:
symbols_with_jun_vals.to_csv(path_or_buf="jema_symbols_with_vals_jun24.csv")

In [10]:
currencies = list(symbols_with_jun_vals[currency_col].unique())

currencies_jun24 = {}
currencies_today = {}

for currency in currencies:
    if currency in currencies_jun24:
        continue
    try:
        data = tv.get_hist(symbol=f'{currency}GBP', exchange='FX_IDC', interval=Interval.in_daily, n_bars=100)
        currencies_jun24[currency] = data.loc['2024-06-30'].close.item()
        currencies_today[currency] = data.iloc[-1].close.item() 
    except:
        print(f"Failed: {currency}GBP")

display(currencies_jun24)
display(currencies_today)

{'SAR': 0.2087,
 'ZAR': 0.04302,
 'GBP': 1.0,
 'QAR': 0.21706,
 'USD': 0.79054,
 'CZK': 0.033726,
 'AED': 0.21482,
 'RUB': 0.009094,
 'HUF': 0.002149,
 'PLN': 0.19634,
 'EUR': 0.84856,
 'TRY': 0.02382,
 'RON': 0.17041,
 'GBX': 0.01}

{'SAR': 0.2015,
 'ZAR': 0.04275,
 'GBP': 1.0,
 'QAR': 0.20957,
 'USD': 0.76387,
 'CZK': 0.033927,
 'AED': 0.2075,
 'RUB': 0.008326,
 'HUF': 0.002167,
 'PLN': 0.19863,
 'EUR': 0.85156,
 'TRY': 0.02214,
 'RON': 0.17091,
 'GBX': 0.01}

In [11]:
for index, row in symbols_with_jun_vals.iterrows():
    currency = row.Currency
    symbols_with_jun_vals.loc[index, 'fx gbp jun 24'] = currencies_jun24[currency]
    symbols_with_jun_vals.loc[index, 'fx gbp today'] = currencies_today[currency]

symbols_with_jun_vals

,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24,Market Value,SP JPM,SP Jun24,SP Today,fx gbp jun 24,fx gbp today
0,THE SAUDI NATIONAL BANK,BSHYYN1,1180,TADAWUL,SAR,1,81271.00,628932.02,7.738702,36.25,36.849998,0.208700,0.201500
1,AL RAJHI BANK COMMON STOCK SAR 10,B12LZH9,1120,TADAWUL,SAR,1,32585.00,559299.16,17.164314,80.00,89.099998,0.208700,0.201500
2,FIRSTRAND LTD COMMON STOCK ZAR 1,6606996,FSR,JSE,ZAR,1,164763.00,548770.63,3.330667,7693.00,8446.000000,0.043020,0.042750
3,JPM GBP LIQUIDITY LVNAV X (DIST.),5819115,GBPGBP,FX_IDC,GBP,1,514330.73,514330.73,1.000000,1.00,1.000000,1.000000,1.000000
4,STANDARD BANK GROUP LTD COMMON STOCK ZAR 10,B030GJ7,SBK,JSE,ZAR,1,47354.00,432459.13,9.132473,21083.00,23642.000000,0.043020,0.042750
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,TATNEFT PJSC TOTAL,NaN,TATN,ALOR,RUB,1,1301425.00,44357.40,0.034084,699.90,603.100000,0.009094,0.008326
98,TCS GROUP HOLDING PLC GDR,0H6380S,TCSG,ALOR,RUB,1,75226.00,1900.14,0.025259,2968.50,2628.000000,0.009094,0.008326
99,VTB BANK PJSC COMMON STOCK RUB 0.01,0H5837S,VTBR,ALOR,RUB,1,4246655.00,39977.21,0.009414,105.70,100.180000,0.009094,0.008326
100,X 5 RETAIL GROUP NV-REGS GDR,0H6365S,FIVE,ALOR,RUB,1,275182.00,1156.81,0.004204,NaN,NaN,0.009094,0.008326


In [31]:
symbols_with_jun_vals.to_csv(path_or_buf="jema_symbols_with_vals_jun24.csv")

In [30]:
for index, row in symbols_with_jun_vals.iterrows():
    fx_rate = row['fx gbp jun 24']
    if row.Currency == 'ZAR':
        fx_rate = fx_rate / 100
    sp_jpm = row[jpm_sp_col]
    sp_jun24 = row[jun24_sp_col]
    sp_jun24_gbp = sp_jun24 * fx_rate
    error = abs(sp_jpm - sp_jun24_gbp) / sp_jpm
    symbols_with_jun_vals.loc[index, 'SP Jun 24 GBP'] = sp_jun24_gbp
    symbols_with_jun_vals.loc[index, 'Error'] = error

symbols_with_jun_vals

,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24,Market Value,SP JPM,SP Jun24,SP Today,fx gbp jun 24,fx gbp today,SP Jun 24 GBP,Error
0,THE SAUDI NATIONAL BANK,BSHYYN1,1180,TADAWUL,SAR,1,81271.00,628932.02,7.738702,36.25,36.849998,0.208700,0.201500,7.565375,0.022397
1,AL RAJHI BANK COMMON STOCK SAR 10,B12LZH9,1120,TADAWUL,SAR,1,32585.00,559299.16,17.164314,80.00,89.099998,0.208700,0.201500,16.696000,0.027284
2,FIRSTRAND LTD COMMON STOCK ZAR 1,6606996,FSR,JSE,ZAR,1,164763.00,548770.63,3.330667,7693.00,8446.000000,0.043020,0.042750,3.309529,0.006346
3,JPM GBP LIQUIDITY LVNAV X (DIST.),5819115,GBPGBP,FX_IDC,GBP,1,514330.73,514330.73,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,0.000000
4,STANDARD BANK GROUP LTD COMMON STOCK ZAR 10,B030GJ7,SBK,JSE,ZAR,1,47354.00,432459.13,9.132473,21083.00,23642.000000,0.043020,0.042750,9.069907,0.006851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,TATNEFT PJSC TOTAL,NaN,TATN,ALOR,RUB,1,1301425.00,44357.40,0.034084,699.90,603.100000,0.009094,0.008326,6.364891,185.742860
98,TCS GROUP HOLDING PLC GDR,0H6380S,TCSG,ALOR,RUB,1,75226.00,1900.14,0.025259,2968.50,2628.000000,0.009094,0.008326,26.995539,1067.745680
99,VTB BANK PJSC COMMON STOCK RUB 0.01,0H5837S,VTBR,ALOR,RUB,1,4246655.00,39977.21,0.009414,105.70,100.180000,0.009094,0.008326,0.961236,101.109097
100,X 5 RETAIL GROUP NV-REGS GDR,0H6365S,FIVE,ALOR,RUB,1,275182.00,1156.81,0.004204,NaN,NaN,0.009094,0.008326,NaN,NaN
